# Day 18

link: https://adventofcode.com/2022/day/18

เราเก็บ coordinate ของแต่ละหยดลาวา โดยเก็บลง Set เพราะเดี๋ยวจะต้องทำ membership test

In [1]:
input = IO.foreach('data/18.txt').to_a.map(&:strip)

cubes = input.map{|line| line.split(',').map(&:to_i)}.to_set
nil

## Part 1

ใน part นี้เราตีความให้ coordinate grid อยู่ที่มุมของลาวา 
หมายความว่า ถ้าโจทย์บอกว่าลาวาอยู่ที่ตำแหน่ง (x, y, z) เราจะตีความว่าลาวานี้อยู่ใน cube ที่อยู่ระหว่าง (x, y, z) ถึง (x+1, y+1, z+1)
ตามรูปนี้ (ให้ chatGPT ช่วยวาดให้)

```
(x,y+1,z+1)+--------+ (x+1,y+1,z+1)
          /        /|
         /        / |
        /        /  |
       +--------+   |
       |        |   + (x+1,y+1,z)
       |        |  /
       |        | /
       |        |/
       +--------+
  (x,y,z)      (x+1,y,z)
```

วิธีการของข้อนี้คือเราจะดูไปทีละ plane สมมติเริ่มจาก XY plane ก่อน ลาวาที่ตำแหน่ง (x, y, z) จะสร้าง 2 surfaces คือสี่เหลี่ยม (x, y), (x+1, y), (x, y+1), (x+1, y+1) ที่ตำแหน่งในแกน Z ที่ z และ z+1 ซึ่งเราสามารถ encode ด้วย (x, y, z) และ (x, y, z+1)

สังเกตว่า แต่ละ surface อาจจะสัมผัสกับลาวา 1 หรือ 2 ก้อนเท่านั้น เราต้องการนับเฉพาะ surface ที่สัมผัสกับ 1 ก้อน ... เรารู้ว่าจากลาวา C ก้อน จะมีทั้งหมด 2C surfaces (เฉพาะใน XY plane) จากนั้นเราก็นับจำนวน D = distinct surfaces ส่วนต่างของสองจำนวนนี้ (2C-D) ก็คือจำนวนของ surface ที่สัมผัสลาวา 2 ก้อน ดังนั้นจำนวน surface ที่สัมผัสลาวา 1 ก้อน ก็คือ 2C - 2(2C-D) = 2D-2C

เราทำแบบเดียวกันนี้กับ XZ plane และ YZ plane ก็จะได้จำนวน surface ที่ต้องการ

In [2]:
x_planes = cubes.flat_map{|x, y, z| [[x, y, z], [x+1, y, z]]}.uniq
y_planes = cubes.flat_map{|x, y, z| [[x, y, z], [x, y+1, z]]}.uniq
z_planes = cubes.flat_map{|x, y, z| [[x, y, z], [x, y, z+1]]}.uniq

puts 2 * (x_planes.size + y_planes.size + z_planes.size) - 6 * cubes.size

3650


## Part 2

part หลังนี้เราใช้ flood fill ใน 3 มิติ จากแต่ละ cell น้ำจะสามารถไหลไปได้ 6 ทิศทาง เราก็จะเช็คว่าปลายทางจะต้องไม่ใช่ตำแหน่งที่มีลาวาอยู่จึงจะไหลไปได้
สังเกตว่าทุกครั้งที่เจอลาวา ก็คือจำนวน surface ที่เราต้องการนั่นเอง

In [3]:
x_min = cubes.map{|x, y, z| x}.min - 1
x_max = cubes.map{|x, y, z| x}.max + 1
y_min = cubes.map{|x, y, z| y}.min - 1
y_max = cubes.map{|x, y, z| y}.max + 1
z_min = cubes.map{|x, y, z| z}.min - 1
z_max = cubes.map{|x, y, z| z}.max + 1


x_size = x_max - x_min + 1
y_size = y_max - y_min + 1
z_size = z_max - z_min + 1

a = Array.new(x_size){ Array.new(y_size) {Array.new(z_size, false)}}
a[0][0][0] = true
q = [[0, 0, 0]]
surface = 0
until q.empty?
  x, y, z = q.pop
  nexts = [[x-1, y, z], [x+1, y, z], [x, y-1, z], [x, y+1, z], [x, y, z-1], [x, y, z+1]]
  nexts.each{|i, j, k|
    next if i < 0 || j < 0 || k < 0 || i >= x_size || j >= y_size || k >= z_size || a[i][j][k]
    if cubes.include?([i + x_min, j + y_min, k + z_min])
      surface += 1
    else
      q.unshift([i, j, k])
      a[i][j][k] = true
    end
  }
end

puts surface

2118
